In [1]:
#importing modules and pacakges
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import csv
import os
import lightgbm as lgb
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
#importing files 
sample_submission=pd.read_csv('/kaggle/input/patient-healthcare/sample_submission.csv')
train=pd.read_csv('/kaggle/input/patient-healthcare/train.csv')
test=pd.read_csv('/kaggle/input/patient-healthcare/test.csv',index_col=False)
#changning categorical data to numrical data so that model can maioulate them
for name in train.select_dtypes(exclude='number').columns:
    le=LabelEncoder()
    #changing categorical data to numerical data
    label = le.fit_transform(train[name])
    #removing the column with categorical columns
    train.drop(name, axis=1, inplace=True)
    #now addig the columns but with numerical data
    train[name] = label
for name in test.select_dtypes(exclude='number').columns:
    le=LabelEncoder()
    #changing categorical data to numerical data
    label = le.fit_transform(test[name])
    #removing the column with categorical columns
    test.drop(name, axis=1, inplace=True)
    #now addig the columns but with numerical data
    test[name] = label
#implementing interpolate function of pandas module to fill null values with  a suitable value 
#limit_direction="forward" means that it will implement the function going down thw table
train=train.interpolate(method='linear',limit_direction='forward')
#limit_direction="backward" means that it will implement the function going up thw table
test=test.interpolate(method='linear',limit_direction='forward')
train_y=train['has_died']
train.pop('has_died')
ids=test['patient_id']
test=np.asarray(test).astype(np.float32)
train=np.asarray(train).astype(np.float32)
train_y=np.asarray(train_y).astype(np.float32)
#paramaters on which model will be hypertuned
rf_grid={
    'n_estimators':np.arange(10,1000,50),
    'max_depth':[None,3,5,10],
    'min_samples_split':np.arange(2,20,2),
    'min_samples_leaf':np.arange(1,20,2)
}
# clf=RandomForestClassifier(n_estimators = 100).fit(train, train_y)
np.random.seed(42)
#implementing randomized search cv in order to hypertune the model
rs_rf=RandomizedSearchCV(RandomForestClassifier(),param_distributions=rf_grid,cv=5,n_iter=20,verbose=True)
#fitting the training data in fit
rs_rf.fit(train, train_y)
#preidcting the results of the test data
arr=rs_rf.predict(test)
#creating a  csv file for putting the result of test data
filename = "records.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(['patient_id','has_died'])
    i=0
    for id in ids:
        csvwriter.writerows([[id],[arr[i]]])
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [2]:
len(arr)
pd.read_csv(filename)

,patient_id,has_died
0,112951.0,NaN
1,0.0,NaN
2,17307.0,NaN
3,0.0,NaN
4,123759.0,NaN
...,...,...
55023,0.0,NaN
55024,100290.0,NaN
55025,0.0,NaN
55026,105358.0,NaN


In [3]:
record=pd.read_csv('records.csv')
recording=record[record['patient_id'].eq(0.0)].index
record.drop(recording, inplace=True)
record.drop(labels='has_died', axis="columns", inplace=True)
record['has_died']=arr
record.to_csv('/kaggle/working/records.csv', index=False)

In [4]:
print(len([1 for i in arr if i==1]))

903
